---

<h1 align="center">Covid 19 - Risk Prediction</h1>
<h3 align="center">Alessandro Sebastiano Catinello</h3>
<h5 align="center">??? 2023</h5>

L'obiettivo principale di questo progetto è mostrare l'implementazione di un **classificatore binario** che, dati i sintomi attuali, lo stato e la storia clinica di un paziente che ha contratto il Covid-19, sia capace di predire se il paziente è a rischio di sviluppare una forma grave della malattia o meno. 

Inizialmente eseguiremo una prima fase di **data cleanining** per testare la qualità dei dati e, qualora fosse necessario, modificarli per renderli più adatti allo studio. 

Successivamente, dopo aver effetuato una breve **analisi esplorativa** sui dati, si testeranno diversi **classificatori**. A tale fine utilizzeremo:
- K-Nearest Neighbors (KNN)
- Support Vector Machine (SVM)
- Naive Bayes (Normal e Multinomial)
- Decision Trees e Random Forest
- Multilayer Perceptron (MLP)

Per ogni algoritmo testeremo vari iperparametri e ne valuteremo le prestazioni, effettueremo inoltre un **confronto** tra i diversi classificatori.

<br>

<br>

**TODO: CURVE DI SOPRAVVIVENZA ???**

---
<h3>Indice</h3>


1. [Configurazione dell'ambiente](#sec1)
2. [Presentazione Dataset e conversione](#sec2)
3. [Data Cleaning](#sec3)
4. [Analisi Dati - EDA](#sec4)
5. 

---

<a id="sec1"></a>
## 1. Configurazione dell'ambiente


In [1]:
library(arrow)

cleanup <- function() {
    rm(list = ls())
    invisible(gc())
}


Caricamento pacchetto: ‘arrow’


Il seguente oggetto è mascherato da ‘package:utils’:

    timestamp




--- 
<a id="sec2"></a>

## 2. Presentazione Dataset e conversione

Il dataset utilizzato è fornito dal governo Messicano ([link](https://www.gob.mx/salud/documentos/datos-abiertos-152127)) e contiene informazioni di pazienti anonimi, comprese le condizioni pregresse, riguardanti l'infezione da SARS-CoV-2 o, più comunemente, Covid-19. Il Dataset contiene circa 4 milioni di pazienti univoci ed è diviso in 24 colonne. Nelle colonne **booleane** troveremo 1 = "SI", 2 = "NO" e 97, 98, 99 = "NON DISPONIBILE":

1. **Patient ID**: ID univoco identificativo del paziente
   
2. **Sex**: Sesso del paziente (1 = Femmina, 2 = Maschio).
3. **Age**: Età del paziente.
4. **Patient type**: Tipo di cura che il paziente ha ricevuto. 1 = "CASA", 2 = "OSPEDALIZZATO".
5. **Admission date**: Data di ammissione del paziente.
6. **Symptoms date**: Data in cui il paziente ha iniziato a manifestare i sintomi.
7. **Death date**: In caso di decesso, ne indica la data; altrimenti troveremo "9999-99-99".
8. **Pneumonia**: Indica se il paziente fosse già affetto da polmonite.
9.  **Pregnancy**: Indica se il paziente fosse in gravidanza.
10. **Diabetes**: Indica se il paziente fosse affetto da diabete.
11. **Copd**: Indica se il paziente fosse affetto da malattie polmonari ostruttive croniche.
12. **Ashtma**: Indica se il paziente fosse affetto da asma.
13. **Inmsupr**: Indica se il paziente fosse immunosoppresso.
14. **Hypertension**: Indica se il paziente fosse affetto da ipertensione.
15. **Cardiovascular**: Indice se il paziente fosse affetto da malattie relative al cuore o ai vasi sanguigni. (Sistema Cardiovascolare)
16. **Renal Chronic**: Indica se il paziente fosse affetto da malattie renali croniche.
17. **Other Desease**: Indica se il paziente fosse affetto da altre malattie.
18. **Obesity**: Indica se il paziente fosse affetto da obesità.
19. **Tobacco**: Indica se il paziente fosse fumatore.
20. **USMER**: Indica se il paziente è stato trattato da un'unità medica nazionale
21. **Origin Country**: Indica il paese di origine del paziente.
22. **Medical Unit**: Indica il tipo di unità medica del Sistema Medico Nazionale che ha curato il paziente.
23. **Intubed**: Indica se il paziente è stato intubato. 
24. **ICU**: Indica se il paziente è stato ricoverato in terapia intensiva.

Data la mole del Dataset, il formato "csv" nativo non è ottimale per le prestazioni e il salvataggio dei dati. Per questo motivo, convertiremo il dataset in un formato più performante, in particolare in un formato "parquet". Questo formato è ottimale per i dataset di grandi dimensioni e permette di salvare i dati in un formato compresso, che permette di ridurre il tempo di caricamento e di salvataggio dei dati. 

A tale fine utilizzeremo la libreria R "arrow".

In [2]:
df <- read.csv("covidOriginal.csv")
write_parquet(df, "covidOriginal.parquet")
cleanup()

---
<a id="seg3"></a>

## 3. Data Cleaning

### 3.1. Caricamento del Dataset

In [3]:
covid <- data.frame(read_parquet("covidOriginal.parquet"))

head(covid)

,PATIENT_ID,USMER,MEDICAL_UNIT,SEX,PATIENT_TYPE,ADMISSION_DATE,SYMPTOMS_DATE,DEATH_DATE,INTUBED,PNEUMONIA,⋯,ASTHMA,INMUSUPR,HYPERTENSION,OTHER_DISEASE,CARDIOVASCULAR,OBESITY,RENAL_CHRONIC,TOBACCO,ORIGIN_COUNTRY,ICU
,<chr>,<int>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<chr>,<int>
1,z4d6fe,1,6,1,1,2020-12-06,2020-11-30,9999-99-99,97,2,⋯,2,2,2,2,2,2,2,2,97,97
2,z3bf80,2,12,2,1,2020-07-28,2020-07-20,9999-99-99,97,2,⋯,2,2,2,2,2,2,2,2,97,97
3,z54912,1,12,1,1,2020-06-12,2020-06-10,9999-99-99,97,2,⋯,2,2,2,2,2,2,2,2,97,97
4,z552ac,1,12,1,1,2020-06-02,2020-05-30,9999-99-99,97,2,⋯,2,2,2,2,2,2,2,2,97,97
5,z59345,1,12,2,1,2020-07-01,2020-06-30,9999-99-99,97,2,⋯,2,2,2,2,2,2,2,2,97,97
6,z4887b,2,12,1,1,2020-08-07,2020-08-05,9999-99-99,97,2,⋯,2,2,1,2,2,2,2,2,97,97


### 3.2. Conversioni ovvie

Osservando i dati notiamo che nella colonna **PREGNANT** i valori per gli utenti maschi sono tutti 97, 98, 99. Ovviamente però non è possibile che un uomo sia in gravidanza. Per questo motivo, sostituiamo questi valori con 2, che indica "NO".

In [4]:
covid$PREGNANT[covid$SEX == 2] <- 2

### 3.3. Eliminazione righe con valori mancanti e conversione in valori booleani

Andremo ora a selezionare le colonne booleane, in queste andremo a eliminare le righe con valori mancanti (97, 98, 99) e infine convertiremo i valori rimanenti in valori booleani. Creeremo inoltre una colonna aggiuntive "**DIED**" per salvare il valore booleano di "Date Died" se diverso da "9999-99-99".

Otterremo infine quindi nella colonna "**SEX**" i valori 1 = "Femmina", 2 = "Maschio" e nelle altre colonne booleane i valori 1 = "SI", 2 = "NO".

In [5]:
columns <- colnames(covid)
remove <- c("PATIENT_ID", "USMER", "SYMPTOMS_DATE",
            "MEDICAL_UNIT", "ADMISSION_DATE", "PATIENT_TYPE",
            "DEATH_DATE", "AGE", "ORIGIN_COUNTRY")
columns <- columns[!(columns %in% remove)]

for (col in columns){
    covid <- covid[!(covid[[col]] %in% c(97, 98, 99)), ]
    covid[[col]] <- covid[[col]] == 1
}
covid$DIED <- covid$DEATH_DATE != "9999-99-99"
covid$DEATH_DATE <- as.Date(covid$DEATH_DATE, format = "%Y-%m-%d")
covid$SYMPTOMS_DATE <- as.Date(covid$SYMPTOMS_DATE, format = "%Y-%m-%d")
covid$ADMISSION_DATE <- as.Date(covid$ADMISSION_DATE, format = "%Y-%m-%d")

### 3.4. Colonna "AT_RISK"

Data la classificazione da eseguire, creeremo una colonna "**AT_RISK**" che indica se il paziente, una volta contratto il covid, è a rischio di imbattersi in complicanze. Per fare ciò utilizzeremo le colonne "**DIED**", "**INTUBED**" e "**ICU**" in quanto sono le complicanze peggiori che possono verificarsi, e ne effettueremo l'OR logico.

Salviamo infine il dataset pulito in un file parquet.

In [6]:
covid$AT_RISK <- covid$DIED | covid$INTUBED | covid$ICU
write_parquet(covid, "covidClean.parquet")


dim(covid)
summary(covid)
cleanup()

[1] 610751     26

  PATIENT_ID            USMER        MEDICAL_UNIT       SEX         
 Length:610751      Min.   :1.000   Min.   : 1.000   Mode :logical  
 Class :character   1st Qu.:1.000   1st Qu.: 4.000   FALSE:358141   
 Mode  :character   Median :1.000   Median : 4.000   TRUE :252610   
                    Mean   :1.467   Mean   : 7.069                  
                    3rd Qu.:2.000   3rd Qu.:12.000                  
                    Max.   :2.000   Max.   :13.000                  
                                                                    
  PATIENT_TYPE ADMISSION_DATE       SYMPTOMS_DATE          DEATH_DATE        
 Min.   :2     Min.   :2020-02-27   Min.   :2020-02-22   Min.   :2020-03-18  
 1st Qu.:2     1st Qu.:2020-08-13   1st Qu.:2020-08-08   1st Qu.:2020-08-26  
 Median :2     Median :2020-12-30   Median :2020-12-25   Median :2021-01-10  
 Mean   :2     Mean   :2021-01-07   Mean   :2021-01-02   Mean   :2021-01-14  
 3rd Qu.:2     3rd Qu.:2021-05-31   3rd Qu.:2021-05-25   3

---


AAAAAAAAAAAAA